In [38]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [39]:
df_greek = pd.read_csv('/Users/bryanburzon/School/DS207/mids-207-final-project-summer23-Burzon-Brown-Caldera-Gomez/icr-identify-age-related-conditions/greeks.csv')
df_train = pd.read_csv('/Users/bryanburzon/School/DS207/mids-207-final-project-summer23-Burzon-Brown-Caldera-Gomez/icr-identify-age-related-conditions/train.csv')


In [40]:
df_greek['Epsilon'].replace('Unknown', np.nan, inplace=True)
df_greek

,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,NaN
2,013f2bd269f5,A,C,M,B,NaN
3,043ac50845d5,A,C,M,B,NaN
4,044fb8a146ec,D,B,F,B,3/25/2020
...,...,...,...,...,...,...
612,fd3dafe738fd,A,B,M,B,9/13/2020
613,fd895603f071,A,B,M,B,9/8/2020
614,fd8ef6377f76,A,C,M,B,7/24/2019
615,fe1942975e40,A,C,M,B,1/31/2019


# Merge both DF

In [41]:
df =  pd.merge(df_train, df_greek, on='Id')
len(df)

617

# Get only class label with greek 

In [42]:
df_class_greek = df[['Id', 'Class', 'Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon' ]]

class_1 = df_class_greek[df_class_greek['Class']==1]


In [43]:
unique_values = set(class_1[['Alpha', 'Beta', 'Gamma', 'Delta']].values.flatten())
unique_values

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'}

In [44]:
unique_values = class_1.groupby(['Alpha', 'Beta', 'Gamma', 'Delta']).size().reset_index().drop_duplicates()

unique_values

,Alpha,Beta,Gamma,Delta,0
0,B,A,H,A,2
1,B,B,H,A,10
2,B,B,H,B,20
3,B,C,G,C,2
4,B,C,G,D,6
5,B,C,H,B,4
6,B,C,H,C,12
7,B,C,H,D,5
8,D,A,F,A,1
9,D,B,E,A,4


In [45]:
class_0 = df_class_greek[df_class_greek['Class']==0]

unique_values = class_0.groupby(['Alpha', 'Beta', 'Gamma', 'Delta']).size().reset_index().drop_duplicates()

unique_values

,Alpha,Beta,Gamma,Delta,0
0,A,B,M,A,51
1,A,B,M,B,97
2,A,C,M,B,242
3,A,C,M,C,44
4,A,C,M,D,11
5,A,C,N,A,1
6,A,C,N,B,63


In [48]:
df_greek['Epsilon'].unique()

<DatetimeArray>
['2019-03-19 00:00:00',                 'NaT', '2020-03-25 00:00:00',
 '2019-10-01 00:00:00', '2019-05-29 00:00:00', '2019-04-24 00:00:00',
 '2019-02-18 00:00:00', '2020-06-19 00:00:00', '2020-06-23 00:00:00',
 '2019-12-18 00:00:00',
 ...
 '2018-01-23 00:00:00', '2020-08-15 00:00:00', '2018-12-26 00:00:00',
 '2014-07-24 00:00:00', '2018-11-21 00:00:00', '2019-11-12 00:00:00',
 '2019-02-13 00:00:00', '2020-01-23 00:00:00', '2020-02-12 00:00:00',
 '2019-07-24 00:00:00']
Length: 198, dtype: datetime64[ns]

In [47]:
df_greek['Epsilon'] =  pd.to_datetime(df_greek['Epsilon'])